In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import requests
from io import StringIO

In [3]:
# Función para calcular RSI
def rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [4]:
# URL de la página de Wikipedia del S&P 500
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Descargar el contenido de la página
response = requests.get(url)

# Convertir el contenido a un objeto StringIO
html_content = StringIO(response.text)

# Leer las tablas de la página
tables = pd.read_html(html_content)

# La tabla de los componentes del S&P 500 suele ser la primera
sp500_table = tables[0]

# Extraer los tickers
sp500_tickers = sp500_table["Symbol"].tolist()

In [5]:
data = yf.download(sp500_tickers[:50], period='3mo', interval='1d', group_by='ticker')

[*********************100%%**********************]  50 of 50 completed


In [23]:
data.columns.get_level_values('Ticker').unique().tolist()

['ACN',
 'A',
 'ADI',
 'APA',
 'GOOGL',
 'ACGL',
 'AMGN',
 'MO',
 'AFL',
 'APTV',
 'AMP',
 'ARE',
 'ALGN',
 'ADSK',
 'AMCR',
 'AKAM',
 'AES',
 'AME',
 'AEP',
 'ABNB',
 'AON',
 'AXP',
 'AMAT',
 'APH',
 'AMZN',
 'AAPL',
 'AJG',
 'LNT',
 'AIG',
 'ABT',
 'AEE',
 'GOOG',
 'ATO',
 'AMT',
 'MMM',
 'ANET',
 'ANSS',
 'ALLE',
 'ALB',
 'AMTM',
 'AIZ',
 'ADM',
 'ADBE',
 'APD',
 'AMD',
 'AWK',
 'AOS',
 'ABBV',
 'ALL',
 'T']

In [18]:
rsi_values = []
for ticker in sp500_tickers[:500]:
    rsi_ = rsi(data['Close'][ticker])
    latest_rsi = rsi_.iloc[-1]  # RSI más reciente
    rsi_values.append({"Ticker": ticker, "RSI": latest_rsi})
rsi_df = pd.DataFrame(rsi_values)

In [19]:
# Crear el heatmap con Plotly
fig = px.scatter(
    rsi_df,
    x="Ticker",
    y="RSI",
    color="RSI",
    color_continuous_scale=["blue", "green", "yellow", "red"],
    title="RSI Heatmap del S&P 500",
    labels={"RSI": "RSI Diario"}
)

# Mostrar el gráfico
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed